In [1]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm

In [2]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [3]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels


In [4]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [5]:
print(train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [6]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [7]:
print(test)
print(test['image'])

                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  images/test\surprise\9853.jpg  surprise
7064  images/test\surprise\9878.jpg  surprise
7065   images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]
0         images/test\angry\10052.jpg
1         images/test\angry\10065.jpg
2         images/test\angry\10079.jpg
3         images/test\angry\10095.jpg
4         images/test\angry\10121.jpg
                    ...              
7061    images/test\surprise\9806.jpg
7062    images/test\surprise\9830.jpg
7063    images/test\surprise\9853.jpg
7064    images/test\surprise\9878.jpg
7065     images/test\surprise\993.jpg
Name:

In [8]:
from tqdm.notebook import tqdm

In [9]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale')
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [10]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [11]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [12]:
x_train = train_features/255.0
x_test = test_features/255.0

In [16]:
from sklearn.preprocessing import LabelEncoder


In [17]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [18]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [19]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [23]:
model = Sequential()
#convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation = 'relu', input_shape = (48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, kernel_size=(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
#fully connected layers
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))
#output layer
model.add(Dense(7, activation = 'softmax'))

In [27]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [28]:
model.fit(x=x_train, y=y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test))

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 122s 531ms/step - accuracy: 0.2369 - loss: 1.8345 - val_accuracy: 0.2628 - val_loss: 1.7932
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 121s 536ms/step - accuracy: 0.2574 - loss: 1.7912 - val_accuracy: 0.3105 - val_loss: 1.7084
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 126s 558ms/step - accuracy: 0.3134 - loss: 1.6920 - val_accuracy: 0.4188 - val_loss: 1.5393
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 127s 562ms/step - accuracy: 0.3713 - loss: 1.5849 - val_accuracy: 0.4486 - val_loss: 1.4466
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 127s 561ms/step - accuracy: 0.4285 - loss: 1.4910 - val_accuracy: 0.4788 - val_loss: 1.3818
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 117s 517ms/step - accuracy: 0.4458 - loss: 1.4403 - val_accuracy: 0.5059 - val_loss: 1.3079
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 121s 534ms/step - accuracy: 0.4602 - loss: 1.4035 - val_accuracy: 0.5004 - val_loss: 1.3220
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 117s 515ms/step - accuracy: 

KeyboardInterrupt: 

In [29]:
model_json = model.to_json()
with open("facialemotionmodel.json",'w') as json_file:
    json_file.write(model_json)
model.save("facialemotionmodel.h5")

In [30]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [33]:
def ef(image):
    img = load_img(image,color_mode='grayscale')
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [37]:
image = 'images/train/angry/22.jpg'
print("orignal image angry")
img = ef(image)
pred = model.predict(img)
pred_label =label[pred.argmax()]
print("model prediction is ",pred_label)

orignal image angry
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
model prediction is  angry


In [39]:
image = 'images/train/angry/27.jpg'
print("orignal image angry")
img = ef(image)
pred = model.predict(img)
pred_label =label[pred.argmax()]
print("model prediction is ",pred_label)

orignal image angry
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
model prediction is  angry
